In [ ]:

import os
import json
import subprocess

def run_solgrep(file_path):
    print ("get data", file_path)
    file_name = file_path.split('/')[-1]
    command = ['./solgrep_wrapper.sh', file_path, "\"modifier.name\"", file_path.replace(file_name, f"solgrep_modifier_data.json")]
    result = subprocess.run(command,check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    command = ['./solgrep_wrapper.sh', file_path, "\"function.name\"", file_path.replace(file_name, f"solgrep_function_data.json")]
    result = subprocess.run(command,check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

def prepare_solgrep_data(metadata_path):
    with open(metadata_path, 'r') as f:
        metadata = json.load(f)
    base_path  = os.path.dirname(os.path.abspath(metadata_path))
    print (len(metadata))
    for meta_k,meta_val in metadata.items():
        file_path = os.path.join(base_path,meta_val.get("file"))
        run_solgrep(file_path)
        # break

json_path = '../smartagent-dataset/Real-Hacks-SunWeb3Sec/access-control/all_files.json'
prepare_solgrep_data(json_path)


In [ ]:
import json

def check_modifiers(json_path):
    modifier_stats = []
    max_modifiers = 0
    max_modifier_key  = None
    with open(json_path, 'r') as f:
        metadata = json.load(f)
    base_path  = os.path.dirname(os.path.abspath(json_path))
    for meta_k,meta_val in metadata.items():
        file_path = os.path.join(base_path,meta_val.get("file"))
        file_name = file_path.split('/')[-1]
        modifier_file = file_path.replace(file_name, f"solgrep_modifier_data.json")
        with open(modifier_file, 'r') as f:
            total_modifiers_len = 0
            modifier_data = json.load(f)

            if len(modifier_data) < 1:
                continue
            # print (modifier_data)
            value = list(modifier_data.values())[0]
            for item in value:
                loc = item.get("loc")
                if loc:
                    total_modifiers_len = total_modifiers_len + (loc[2]-loc[0])
        if total_modifiers_len > max_modifiers:
            max_modifiers = total_modifiers_len
            max_modifier_key = meta_k
        modifier_stats.append(total_modifiers_len)
    print (max_modifiers, max_modifier_key)
    return modifier_stats
json_path = '../smartagent-dataset/AccessControl/SmartBugsWild/all_files.json'
modifier_stats = check_modifiers(json_path)
print
print (modifier_stats)
import matplotlib.pyplot as plt
plt.hist(modifier_stats)